# Nulls e Expressões CASE

Nesta seção, abordaremos valores `NULL` e expressões `CASE`. Um valor `NULL` não representa nenhum valor, assim como `None` ou `NaN` em Python indicam um valor em branco. A expressão `CASE` nos permite associar condições a valores resultantes, assim como `if`/`elif` em Python.

Abordaremos ambas as operações em SQL.

## Configuração
Primeiro, faça a configuração. Baixe o arquivo de banco de dados SQLite `company_operations.db` e conecte-se a ele. Também inclua `pandas` para exibir os resultados da nossa consulta SQL como um `DataFrame`.

In [ ]:
import sqlite3
import pandas as pd
import urllib.request

# baixe o banco de dados SQLite e conecte-se a ele
urllib.request.urlretrieve("https://github.com/thomasnield/anaconda_intro_to_sql/blob/main/company_operations.db?raw=true", "company_operations.db")
conn = sqlite3.connect('company_operations.db')

## Valores NULL

Vamos dar uma olhada na tabela `WEATHER_MONITOR`. Veja estes quatro registros.

In [ ]:
sql = """
SELECT * FROM WEATHER_MONITOR 
WHERE REPORT_CODE IN ('LJVE08D', 'EP4AKZR', '1FC27OH', 'F4DEAK3') 
"""

pd.read_sql(sql, conn)


Observe como algumas colunas têm valores `NaN` ou `None`, que indicam um valor `NULL`. Um valor nulo está em branco, o que significa que nenhum valor foi fornecido (não confundir com `0`, que é um valor ou uma string vazia `''`).

Observe que os bancos de dados SQL terão `NULL` para valores em branco, mas o Pandas os reinterpretará como `None` ou `NaN`, dependendo se a coluna for numérica ou não.

Se tivermos valores nulos para chuva, isso pode indicar que os registros de chuva não foram possíveis porque os instrumentos estavam quebrados. O mesmo vale para `SNOW` e outros campos que permitem valor nulo.

Para qualificar um valor nulo, use `IS NULL`. Abaixo, encontramos registros sem uma medição de `RAIN` registrada.

In [ ]:
sql = """
SELECT * FROM WEATHER_MONITOR 
WHERE RAIN IS NULL 
"""

pd.read_sql(sql, conn)


Para qualificar registros que não são nulos, qualifique com `IS NOT NULL`.

In [ ]:
sql = """
SELECT * FROM WEATHER_MONITOR 
WHERE RAIN IS NOT NULL 
"""

pd.read_sql(sql, conn)


Observe que, se você não manipular valores `NULL` explicitamente na condição `WHERE` em uma determinada coluna, os valores `NULL` serão sempre omitidos. Por exemplo, se qualificarmos para registros onde `RAIN > 0`, os valores `NULL` serão omitidos.

In [ ]:
sql = """
SELECT * FROM WEATHER_MONITOR 
WHERE RAIN > 0 
"""

pd.read_sql(sql, conn)


Se você quiser incluir valores `NULL` em sua condição, permita explicitamente `NULL`.

In [ ]:
sql = """
SELECT * FROM WEATHER_MONITOR 
WHERE RAIN IS NULL OR RAIN > 0 
"""

pd.read_sql(sql, conn)


Uma função útil para se saber de cor é `COALESCE()`. Ela receberá um valor possivelmente `NULL` e o converterá para um valor diferente se for de fato `NULL`. Caso contrário, deixará o valor inalterado.

O primeiro argumento para `COALESCE()` é o valor que pode ser `NULL`. O segundo argumento é o valor para o qual será convertido, caso seja de fato `NULL`. Podemos tratar todos os valores `RAIN` que são `NULL` como `0` em `COALESCE()` abaixo.

In [ ]:
sql = """
SELECT * FROM WEATHER_MONITOR 
WHERE COALESCE(RAIN,0) > 0 
"""

pd.read_sql(sql, conn)


Como outro exemplo, para transformar valores `RAIN` ausentes em `-1`, podemos usar `COALESCE` assim.

In [ ]:
sql = """
SELECT REPORT_CODE, 
RAIN, 
COALESCE(RAIN,-1) AS COALESCED_RAIN 

FROM WEATHER_MONITOR 
WHERE REPORT_CODE IN ('G0UINBG', 'PJVNOSP')
"""

pd.read_sql(sql, conn)


## Expressão CASE

Dê uma olhada no campo `TEMPERATURE` na tabela.

In [ ]:
sql = """
SELECT REPORT_CODE, TEMPERATURE
FROM WEATHER_MONITOR
"""

pd.read_sql(sql, conn)


Digamos que quiséssemos categorizar cada temperatura como `HOT`, `MILD` ou `COLD`. Para isso, teríamos que usar uma expressão `CASE` e anexar uma condição a cada rótulo. Vamos demonstrar:

In [ ]:
sql = """
SELECT REPORT_CODE, 
TEMPERATURE,

CASE 
  WHEN TEMPERATURE >= 78 THEN 'HOT'
  WHEN TEMPERATURE >= 60 THEN 'MILD'
  ELSE 'COLD'
END AS TEMPERATURE_LABEL

FROM WEATHER_MONITOR
"""

pd.read_sql(sql, conn)


Observe como usamos um `CASE` para abrir a expressão `CASE`. Cada `WHEN` especifica uma condição e `THEN` especifica o valor resultante se essa condição for verdadeira. Cada condição é avaliada de cima para baixo, e a primeira que for considerada verdadeira será escolhida. Um `ELSE` pode ser opcionalmente anexado para especificar um valor padrão se todas as outras condições não forem atendidas. Nesse caso, definimos qualquer outro registro como `COLD`, pois já deduzimos que ele não é `HOT` ou `MILD`.

No entanto, você deve ter cuidado com valores `NULL` se eles estiverem presentes em uma coluna. Se você usar um `ELSE` no campo `TEMPERATURE` e esse campo tiver valores `NULL` (são três), eles serão rotulados como `NULL`. Uma maneira melhor de lidar com os valores `NULL` pode ser ter uma condição explícita para `COLD` e então tornar o `ELSE` o termo geral para anomalias como `NULL` e rotulá-los como `N/A`.

In [ ]:
sql = """
SELECT REPORT_CODE, 
TEMPERATURE,

CASE 
  WHEN TEMPERATURE >= 78 THEN 'HOT'
  WHEN TEMPERATURE >= 60 THEN 'MILD'
  WHEN TEMPERATURE < 60 THEN 'COLD'
  ELSE 'N/A'
END AS TEMPERATURE_LABEL

FROM WEATHER_MONITOR
"""

pd.read_sql(sql, conn)


Com uma expressão `CASE`, agora você pode fazer agregações mais interessantes em campos que antes não estavam disponíveis. Por exemplo, podemos obter uma `COUNT` do número de registros divididos por `TEMPERATURE_LABEL`.

In [ ]:
sql = """
SELECT 

CASE 
  WHEN TEMPERATURE >= 78 THEN 'HOT'
  WHEN TEMPERATURE >= 60 THEN 'MILD'
  WHEN TEMPERATURE < 60 THEN 'COLD'
  ELSE 'N/A'
END AS TEMPERATURE_LABEL,

COUNT(*) AS RECORD_COUNT

FROM WEATHER_MONITOR

GROUP BY TEMPERATURE_LABEL
"""

pd.read_sql(sql, conn)


A propósito, você deve ter percebido que `COALESCE` é uma abreviação da expressão `CASE` para converter valores `NULL`. Veja nosso exemplo anterior mostrando os valores `RAIN` unidos.

In [ ]:
sql = """
SELECT REPORT_CODE, 
RAIN, 
COALESCE(RAIN,-1) AS COALESCED_RAIN 

FROM WEATHER_MONITOR 
WHERE REPORT_CODE IN ('G0UINBG', 'PJVNOSP')
"""

pd.read_sql(sql, conn)


Podemos expressar isso usando uma expressão `CASE`.

In [ ]:
sql = """
SELECT REPORT_CODE, 
RAIN, 
CASE WHEN RAIN IS NULL THEN -1 ELSE RAIN END AS COALESCED_RAIN 

FROM WEATHER_MONITOR 
WHERE REPORT_CODE IN ('G0UINBG', 'PJVNOSP')
"""

pd.read_sql(sql, conn)


## O truque do caso NULL

Digamos que você calcule a chuva total dividida por `YEAR` e `MONTH`, apenas para o `YEAR` de 2021.

In [ ]:
sql = """
SELECT 
CAST(strftime('%Y', REPORT_DATE) AS INTEGER) AS YEAR, 
CAST(strftime('%m', REPORT_DATE) AS INTEGER) AS MONTH, 

SUM(RAIN) AS TOTAL_RAIN

FROM WEATHER_MONITOR 

GROUP BY YEAR, MONTH
"""

pd.read_sql(sql, conn)


Agora você quer dividir a coluna `TOTAL_RAIN` em duas colunas, uma para quando um `TORNADO` estiver presente e outra para quando não estiver. Qual é o problema aqui?

In [ ]:
sql = """
SELECT 
CAST(strftime('%Y', REPORT_DATE) AS INTEGER) AS YEAR, 
CAST(strftime('%m', REPORT_DATE) AS INTEGER) AS MONTH, 

SUM(RAIN) AS TOTAL_TORNADO_RAIN,
SUM(RAIN) AS TOTAL_NON_TORNADO_RAIN

FROM WEATHER_MONITOR 

WHERE TORNADO = 1 
AND YEAR = 2021

GROUP BY YEAR, MONTH
"""

pd.read_sql(sql, conn)


Essa condição `WHERE` inconvenientemente obriga você a `TORNADO` ser 1 ou 0, mas não ambos, para cada coluna. Mas você pode contornar isso usando uma expressão `CASE` e inserindo as respectivas condições. Observe abaixo como interceptamos os valores que entram em cada `SUM()` verificando a condição `TORNADO` e, se falhar, adicionamos um `0` a `SUM`. Inteligente, não é?

In [ ]:
sql = """
SELECT 
CAST(strftime('%Y', REPORT_DATE) AS INTEGER) AS YEAR, 
CAST(strftime('%m', REPORT_DATE) AS INTEGER) AS MONTH, 

SUM(CASE WHEN TORNADO = 1 THEN RAIN ELSE 0 END) AS TOTAL_TORNADO_RAIN,
SUM(CASE WHEN TORNADO = 0 THEN RAIN ELSE 0 END) AS TOTAL_NON_TORNADO_RAIN

FROM WEATHER_MONITOR 

WHERE YEAR = 2021 

GROUP BY YEAR, MONTH
"""

pd.read_sql(sql, conn)


No entanto, um `0` para a condição falsa pode ser problemático para outras operações de agregação como `MIN`, `MAX`, `AVG` e `COUNT`, pois afetará esses cálculos, ao contrário de `SUM`. Você pode usar `NULL`, pois ele será ignorado por todos os operadores de agregação, incluindo `SUM`.

In [ ]:
sql = """
SELECT 
CAST(strftime('%Y', REPORT_DATE) AS INTEGER) AS YEAR, 
CAST(strftime('%m', REPORT_DATE) AS INTEGER) AS MONTH, 

SUM(CASE WHEN TORNADO = 1 THEN RAIN ELSE NULL END) AS AVG_TORNADO_RAIN,
SUM(CASE WHEN TORNADO = 0 THEN RAIN ELSE NULL END) AS AVG_NON_TORNADO_RAIN

FROM WEATHER_MONITOR 

WHERE YEAR = 2021 

GROUP BY YEAR, MONTH
"""

pd.read_sql(sql, conn)


Poucas pessoas que usam `SQL` conhecem esse truque, que pode evitar muitas consultas e tabelas derivadas confusas. Use-o com moderação!

## EXERCÍCIO

Para cada `LOCATION_ID`, calcule a precipitação total do ano anterior `PY_RAIN` e a precipitação total do ano atual `CY_RAIN`. Substitua os pontos de interrogação `?` e considere que 2021 é o ano atual.

In [ ]:
sql = """
SELECT 

LOCATION_ID,

SUM(
  CASE WHEN CAST(strftime('%Y', REPORT_DATE) AS INTEGER) = ? THEN ? ELSE ? END
) AS CY_RAIN,

SUM(
  CASE WHEN CAST(strftime('%Y', REPORT_DATE) AS INTEGER) = ? THEN ? ELSE ? END
) AS PY_RAIN

FROM WEATHER_MONITOR 

WHERE CAST(strftime('%Y', REPORT_DATE) AS INTEGER) IN (2020, 2021)

GROUP BY LOCATION_ID
"""

pd.read_sql(sql, conn)


### RESPOSTA A BAIXO

|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

In [ ]:
sql = """
SELECT 

LOCATION_ID,

SUM(
  CASE WHEN CAST(strftime('%Y', REPORT_DATE) AS INTEGER) = 2021 THEN RAIN ELSE 0 END
) AS CY_RAIN,

SUM(
  CASE WHEN CAST(strftime('%Y', REPORT_DATE) AS INTEGER) = 2020 THEN RAIN ELSE 0 END
) AS PY_RAIN

FROM WEATHER_MONITOR 

WHERE CAST(strftime('%Y', REPORT_DATE) AS INTEGER) IN (2020, 2021)

GROUP BY LOCATION_ID
"""

pd.read_sql(sql, conn)
